<h1>ML Artificial Neural Network Example</h1>
This example demonstrates classification of time-series data from the force-torque sensor to detect grasping failure. The data is taken from the <a href="https://archive.ics.uci.edu/ml/datasets/Robot+Execution+Failures">Robot execution failures</a> dataset. It consists of 87 labeled instances of sequences of 15 F/T measurements and class labels.

The data is parsed into a pandas dataframe, broken into training and test set, and run through a random forest regressor from sklearn. 


In [113]:
# Import data processing tools
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib ipympl

import numpy as np

# Import ML tools
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras import regularizers
from keras.utils import np_utils

# Import tools needed for visualization
from IPython.display import Image, display


The dataset consists of time series of 6-DoF force/torque measurements that are labeled with success ('normal') or failure ('obstruction', 'collision', 'fr_collision'). The raw data can be inspected <a href="https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data">here</a> and needs to be converted in table format. <br><br>
The code for parsing the data has been adopted from a <a href="https://tsfresh.readthedocs.io/en/latest/_modules/tsfresh/examples/robot_execution_failures.html">tsfresh tutorial</a>. It proceeds by reading the data file line by line and storing an entire time series in a single row. Sucess and the various failure modes are coded as a boolean value. 

In [114]:
    id_to_target = {}
    df_rows = []
    df_block = []

    with open('data/lp1.data') as f:
        cur_id = 0
        time = 0

        for line in f.readlines():
            # New sample --> increase id, reset time and determine target
            if line[0] not in ['\t', '\n']:               
                time = 0
                #id_to_target[cur_id] = line.strip()
                id_to_target[cur_id] = (line.strip()=='normal')
                if(df_block): 
                    df_rows.append(np.array(df_block).reshape(1,90)[0].tolist())
                df_block = []
                cur_id += 1
            # Data row --> split and convert values, create complete df row
            elif line[0] == '\t':
                values = list(map(int, line.split('\t')[1:]))
                #df_rows.append([cur_id, time] + values)
                df_block.append(values)
                time += 1
        df_rows.append(np.array(df_block).reshape(1,90)[0].tolist())

    columns=[]
    for i in range(0,15):
        columns=columns+['F_x'+str(i), 'F_y'+str(i), 'F_z'+str(i), 'T_x'+str(i), 'T_y'+str(i), 'T_z'+str(i)]
    
    df = pd.DataFrame(df_rows, columns=columns) # Store all data in a Pandas dataframe
    y = pd.Series(id_to_target) # Store all class labels in a Pandas series

In [115]:
df.tail()

,F_x0,F_y0,F_z0,T_x0,T_y0,T_z0,F_x1,F_y1,F_z1,T_x1,...,F_z13,T_x13,T_y13,T_z13,F_x14,F_y14,F_z14,T_x14,T_y14,T_z14
83,-29,-10,-208,180,12,-11,-29,-4,-246,192,...,-918,396,101,-23,-105,63,-912,394,100,-25
84,4,6,50,-25,1,0,4,8,44,-26,...,4,-46,14,-7,19,14,2,-42,10,-7
85,21,15,-14,-46,16,-7,23,18,-27,-49,...,-346,-88,154,4,148,69,-411,-95,191,8
86,171,86,-486,-113,222,13,198,96,-566,-125,...,-1036,-118,466,44,342,154,-1036,-118,466,44
87,-6,3,52,-15,-17,3,-7,2,52,-17,...,26,-29,-27,5,-13,2,15,-25,-25,6


In [116]:
y.head()

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [117]:
labels=y
features=np.array(df).astype(int)
feature_list=list(df.columns)

labels=labels.astype(int)
categorical_labels = np_utils.to_categorical(labels, num_classes=2).astype(int)


In [118]:
Y_test.astype(int)

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0]])

In [119]:
X_train, X_test, Y_train, Y_test = train_test_split(features, categorical_labels, test_size = 0.25)

In [120]:
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 2
OPTIMIZER = Adam() #RMSprop() #SGD()
N_HIDDEN = 64
VALIDATION_SPLIT = 0.2

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(90,)))
model.add(Activation('relu'))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
#model.summary()
model.compile(loss='binary_crossentropy',optimizer=OPTIMIZER, metrics=['accuracy'])


In [121]:
history = model.fit(X_train, Y_train, 
                    batch_size=BATCH_SIZE, epochs=NB_EPOCH,verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

Train on 52 samples, validate on 14 samples
Epoch 1/20
52/52 [==============================] - 1s 25ms/step - loss: 4.8518 - acc: 0.6731 - val_loss: 4.9680 - val_acc: 0.6429
Epoch 2/20
52/52 [==============================] - 0s 108us/step - loss: 4.6367 - acc: 0.7115 - val_loss: 4.7170 - val_acc: 0.6429
Epoch 3/20
52/52 [==============================] - 0s 83us/step - loss: 4.6242 - acc: 0.7115 - val_loss: 4.5914 - val_acc: 0.7143
Epoch 4/20
52/52 [==============================] - 0s 98us/step - loss: 4.3295 - acc: 0.7115 - val_loss: 4.5805 - val_acc: 0.7143
Epoch 5/20
52/52 [==============================] - 0s 94us/step - loss: 4.3158 - acc: 0.7308 - val_loss: 4.5801 - val_acc: 0.7143
Epoch 6/20
52/52 [==============================] - 0s 77us/step - loss: 4.3158 - acc: 0.7308 - val_loss: 4.5801 - val_acc: 0.7143
Epoch 7/20
52/52 [==============================] - 0s 88us/step - loss: 4.3158 - acc: 0.7308 - val_loss: 4.5801 - val_acc: 0.7143
Epoch 8/20
52/52 [====================

In [122]:
predictions = np.round(model.predict(X_test))

<h2>Results</h2>
We can now compare the predictions made by the random forest with the known labels from the test set and calculate the percentage of errorneous predictions. 

In [123]:
errors = abs(predictions-Y_test)

In [124]:
print('Error: ',sum(errors)/len(Y_test)*100,'%')

Error:  [13.63636364 13.63636364] %
